# Maps of the Iberian Peninsula

Olivier Defaux(1), Gerd Graßhoff(1,2), Mohammad Yeghaneh(2)

1: Max-Planck-Institute for the History of Science, Berlin; 
2: Humboldt University, Berlin

Date:

Ptolemy's catalogue of localities contains a list of places with their coordinates that enables to draw maps of the Iberian Peninsula. Ptolemy's text came down to us in two versions that are sometimes different regarding the spelling of the toponyms and the values of the coordinates. These versions are usually called "Omega recension" and "Xi recension". Based on the Greek text of Ptolemy's catalogue edited by A. Stückelberger and G. Graßhoff (2006), we will draw Ptolemy's map of the Iberian Peninsula and compare the two transmitted versions of his work.

In [1]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
%load_ext autoreload
%autoreload 2

In [2]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import output_notebook, output_file, save
from bokeh.models import HoverTool,BoxZoomTool,ResetTool, WheelZoomTool
from bokeh.layouts import row, gridplot, layout

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('max_colwidth', 260)

In [4]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro","collapse_phrases":False}

In [5]:
from tools import Geography, reformatCoord,reformatIntFrac, flatten_list,findTS,Js2Geodf

Loading BokehJS ...

## Resources

We selected the part of Ptolemy's catalogue of localities related to the Iberian Peninsula (*Geography* 2.4–6) and we encoded the Greek text of the two recensions (Omega and Xi). The files can be loaded from the open-access plateform Zenodo.

### Import

In [6]:
# import .json file from DOI number

In [7]:
Omega = pd.read_json('../data/OmegaStructure2.json',encoding="utf8")
Xi = pd.read_json('../data/XiStructure2.json',encoding="utf8")

### Transform the .json files into dataframes

The .json files reproduce Ptolemy's text and structure. In order to work with the coordinates in an efficient way, we will transform the file format to produce a simple dataframe.

In [8]:
dfOmega=Js2Geodf(Omega["chapters"][0])
dfXi=Js2Geodf(Xi["chapters"][0])

In [9]:
# Sample of the dataframe for the Omega recension:
dfOmega[24:31]

,ID,category,coord,people,text,toponym,type,type_sec
24,2.04.05.13,city,"{'long': {'integer': 'ς', 'fraction': 'δ'}, 'lat': {'integer': 'λς', 'fraction': 'γο'}}",Turduli,NaN,Βαίλων πόλις,locality,coast section
25,2.04.06.01,NaN,NaN,NaN,Βαστουλῶν τῶν καλουμένων Ποινῶν,NaN,people,coast section
26,2.04.06.02,city,"{'long': {'integer': 'ς', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'L'}}",Bastuli,NaN,Μενραλία,locality,coast section
27,2.04.06.03,city,"{'long': {'integer': 'ς', 'fraction': 'γο'}, 'lat': {'integer': 'λς', 'fraction': 'γ'}}",Bastuli,NaN,Τρανσδούκτα,locality,coast section
28,2.04.06.04,city,"{'long': {'integer': 'ζ', 'fraction': 'δ'}, 'lat': {'integer': 'λς', 'fraction': 'ς'}}",Bastuli,NaN,Βαρβησόλα,locality,coast section
29,2.04.06.05,city,"{'long': {'integer': 'ζ', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'ς'}}",Bastuli,NaN,Καρτμία,locality,coast section
30,2.04.06.06,mountain,"{'long': {'integer': 'ζ', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'δ'}}",Bastuli,NaN,Κάρπη ὅρος καὶ στήλη τῆς ἐντὸς θαλάσσης,locality,coast section


## Transform Greek numbers to decimal coordinates

### Greek numeral notation

Each latitude and each longitude is composed of two parts: an integer part (a whole number of degrees) and a decimal part (written as a fraction of degrees).
In order to plot the localities, we have to transpose the Greek numeral system used by Ptolemy into a modern format, using decimal. The symbols used by Ptolemy to express the coordinates are Greek letters and a special sign for the fraction 1/2.

In [10]:
gfrac={"":0,"ιβ":1/12,"ς":1/6,"δ":1/4,"γ":1/3,"γιβ":5/12,"L":1/2,"Lιβ":7/12,"γο":2/3,"Lδ":3/4,"Lγ":5/6,"Lγιβ":11/12,"η":1/8,"Lς":2/3,"ςL":2/3}
gint={"":0,"α":1,"β":2,"γ":3,"δ":4,"ε":5,"ς":6,"ζ":7,"η":8,"θ":9,"ι":10,"κ":20,"λ":30,"μ":40}

In [11]:
pd.DataFrame(list(gint.items()), columns=['Greek', 'Integers']).sort_values('Integers').set_index('Greek').T

Greek,,α,β,γ,δ,ε,ς,ζ,η,θ,ι,κ,λ,μ
Integers,0,1,2,3,4,5,6,7,8,9,10,20,30,40


In [12]:
pd.DataFrame(list(gfrac.items()), columns=['Greek', 'Decimal']).sort_values('Decimal').set_index('Greek').T

Greek,,ιβ,η,ς,δ,γ,γιβ,L,Lιβ,γο,Lς,ςL,Lδ,Lγ,Lγιβ
Decimal,0.0,0.083333,0.125,0.166667,0.25,0.333333,0.416667,0.5,0.583333,0.666667,0.666667,0.666667,0.75,0.833333,0.916667


### Translate into modern numeral notation

We create two columns for each dataframe: one for the longitude, one for the latitude.

In [13]:
# Omega
dfTemp = dfOmega.copy()
dfTemp['longitude'] = dfOmega.apply(lambda row: reformatCoord(row,'long','coord'),axis=1).apply(reformatIntFrac)
dfTemp['latitude'] = dfOmega.apply(lambda row: reformatCoord(row,'lat','coord'),axis=1).apply(reformatIntFrac)

In [14]:
dfTemp[26:31]

,ID,category,coord,people,text,toponym,type,type_sec,longitude,latitude
26,2.04.06.02,city,"{'long': {'integer': 'ς', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'L'}}",Bastuli,NaN,Μενραλία,locality,coast section,6.500000,36.500000
27,2.04.06.03,city,"{'long': {'integer': 'ς', 'fraction': 'γο'}, 'lat': {'integer': 'λς', 'fraction': 'γ'}}",Bastuli,NaN,Τρανσδούκτα,locality,coast section,6.666667,36.333333
28,2.04.06.04,city,"{'long': {'integer': 'ζ', 'fraction': 'δ'}, 'lat': {'integer': 'λς', 'fraction': 'ς'}}",Bastuli,NaN,Βαρβησόλα,locality,coast section,7.250000,36.166667
29,2.04.06.05,city,"{'long': {'integer': 'ζ', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'ς'}}",Bastuli,NaN,Καρτμία,locality,coast section,7.500000,36.166667
30,2.04.06.06,mountain,"{'long': {'integer': 'ζ', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'δ'}}",Bastuli,NaN,Κάρπη ὅρος καὶ στήλη τῆς ἐντὸς θαλάσσης,locality,coast section,7.500000,36.250000


In [15]:
# Xi
dfTempX = dfXi.copy()
dfTempX['longitude'] = dfXi.apply(lambda row: reformatCoord(row,'long','coord'),axis=1).apply(reformatIntFrac)
dfTempX['latitude'] = dfXi.apply(lambda row: reformatCoord(row,'lat','coord'),axis=1).apply(reformatIntFrac)

In [16]:
example={"integer":dfXi.coord[4]["long"]["integer"],"fraction":dfXi.coord[4]["long"]["fraction"]}
print(example)

{'integer': 'δ', 'fraction': 'ιβ'}


In [17]:
row=dfTemp.iloc[26]
row
a=reformatCoord(row,'lat','coord')
a

('λς', 'L')

In [18]:
reformatIntFrac(a)

36.5

## Data preparation for the map drawing

### Coasts of the peninsula (Omega)

We select the localities that form the coasts of the peninsula. 

In [19]:
dfOmegaCoasts = dfTemp[(dfTemp.type_sec == 'coast section') & (dfTemp.type == 'locality')]
dfOmegaCoasts = dfOmegaCoasts[dfOmegaCoasts.category.apply(lambda row: row not in ['river path','river source'])]

We sort the localities in the correct order.

In [20]:
dfOmegaCoasts = pd.concat([dfOmegaCoasts[39:70].sort_index(ascending=False),dfOmegaCoasts[29:38].sort_index(ascending=False),dfOmegaCoasts[:2],dfOmegaCoasts[4:29],dfTemp[(dfTemp.ID == '2.04.08.08')],dfOmegaCoasts[3:4],dfOmegaCoasts[70:99]])

### Inland boundary lines (Omega)

Ptolemy's map of the Iberian peninsula contains several boundary lines between the provinces that we want to draw.

In [21]:
# boundaries of the Baetica province
boundary_om1 = dfTemp[(dfTemp.ID == '2.04.03.01')].append(dfTemp[(dfTemp.ID == '2.04.03.03')]).append(dfTemp[(dfTemp.ID == '2.04.03.04')]).append(dfTemp[(dfTemp.ID == '2.04.03.07')])
boundary_om1 = boundary_om1[['longitude','latitude']].values

#boundaries of the Lusitania province
boundary_om2 = dfTemp[(dfTemp.ID == '2.05.01.04')].append(dfTemp[(dfTemp.ID == '2.05.01.06')]).append(dfTemp[(dfTemp.ID == '2.05.04.05')]).append(dfTemp[(dfTemp.ID == '2.04.03.04')])
boundary_om2 = boundary_om2[['longitude','latitude']].values

The Pyrenees are defined by four points: three are defined by Ptolemy in the catalogue of the Iberian localities, one is defined in the chapter devoted to the province of Gallia Aquitania.

In [22]:
boundary_om3 = pd.DataFrame({'longitude':[15,17,19,20.333], 'latitude':[45.833,43,43.167,42.333]}).values

### Dictionary for coastlines and boudaries (Omega)

We save the data for the coastline and boundaries as a dictionary for later use.

In [23]:
OmegaCoasts = dfOmegaCoasts[['longitude','latitude']].values

In [24]:
ob_dict={"OmegaCoasts":OmegaCoasts,'boundary_om1' : boundary_om1, 'boundary_om2' : boundary_om2, "boundary_om3":boundary_om3 }

### Coasts of the peninsula (Xi)

In [25]:
dfXiCoasts = dfTempX[(dfTempX.type_sec == 'coast section') & (dfTempX.type == 'locality')]
dfXiCoasts = dfXiCoasts[dfXiCoasts.category.apply(lambda row: row not in ['river path','river source'])]

In [26]:
dfXiCoasts = pd.concat([dfXiCoasts[37:68].sort_index(ascending=False),dfXiCoasts[27:36].sort_index(ascending=False),dfXiCoasts[:2],dfXiCoasts[4:27],dfTempX[(dfTempX.ID == '2.04.08.08')],dfXiCoasts[3:4],dfXiCoasts[70:99]]);

In [27]:
XiCoasts = dfXiCoasts[['longitude','latitude']].values

### Inland boundary lines (Xi)

In [28]:
# boundaries of the Baetica province
boundary_xi1 = dfTempX[(dfTempX.ID == '2.04.03.01')].append(dfTempX[(dfTempX.ID == '2.04.03.03')]).append(dfTempX[(dfTempX.ID == '2.04.03.04')]).append(dfTempX[(dfTempX.ID == '2.04.03.07')])
boundary_xi1 = boundary_xi1[['longitude','latitude']].values

#boundaries of the Lusitania province
boundary_xi2 = dfTempX[(dfTempX.ID == '2.05.01.04')].append(dfTempX[(dfTempX.ID == '2.05.01.06')]).append(dfTempX[(dfTempX.ID == '2.05.04.05')]).append(dfTempX[(dfTempX.ID == '2.04.03.04')])
boundary_xi2 = boundary_xi2[['longitude','latitude']].values

In [29]:
boundary_xi3 = pd.DataFrame({'longitude':[15.167,17,19,20.333], 'latitude':[45.833,43,43.167,42.333]}).values

In [30]:
xb_dict={'XiCoasts':XiCoasts, 'boundary_xi1' : boundary_xi1, 'boundary_xi2' : boundary_xi2, 'boundary_xi3' : boundary_xi3}

## Map of the Iberian peninsula, Omega recension

In [31]:
df_dict={"dfTemp" : dfTemp,  "dfTempX" : dfTempX}

In [32]:
fig=Geography(ob_dict,xb_dict,df_dict)

In [40]:
fig.plot_recension(xb_dict,ob_dict,df_dict,"Omega")

## Map of the Iberian peninsula, Xi recension

In [41]:
fig.plot_recension(ob_dict,xb_dict,df_dict,"Xi")

## Comparison between the two recensions

In [42]:
fig.plot_recension_all(ob_dict,xb_dict,df_dict)

In [43]:
fig.plot_compare_recension(ob_dict,xb_dict,df_dict)